Importing Libraries

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
 # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

import librosa
import IPython.display as ipd
import matplotlib.pyplot as plt
from scipy.io import wavfile
from scipy.fftpack import fft
from scipy import signal
import warnings

warnings.filterwarnings("ignore")

os.listdir('../input/')
import pandas as pd

In [ ]:
import os
print(os.listdir("../input/sr-data/Data"))

**Visualization of Audio signal in time series domain**

In [ ]:
train_audio_path = '../input/sr-data/Data/'
samples, sample_rate = librosa.load(train_audio_path+'cat/01648c51_nohash_0.wav', sr = 16000)
fig = plt.figure(figsize=(14, 8))
ax1 = fig.add_subplot(211)
#ax1.set_title('Raw wave of ' + '../input/cat/00b01445_nohash_0.wav')
ax1.set_xlabel('time')
ax1.set_ylabel('Amplitude')
ax1.plot(np.linspace(0, sample_rate/len(samples), sample_rate), samples)

**Silence Removal**
    Removing the silence from the audios.

In [ ]:
ipd.Audio(samples, rate=sample_rate)

In [ ]:
samples_cut = samples[7000:14000]
ipd.Audio(samples_cut, rate=sample_rate)

But it is hard to remove the silence from all the audios.

In [ ]:
filename = 'bird/01648c51_nohash_0.wav'
new_sample_rate = 8000

sample_rate, samples = wavfile.read(str(train_audio_path) + filename)
resampled = signal.resample(samples, int(new_sample_rate/sample_rate * samples.shape[0]))

In [ ]:
ipd.Audio(samples, rate=sample_rate)

In [ ]:
samples

In [ ]:
ipd.Audio(resampled, rate=new_sample_rate)

**Data Pre processing**

In [ ]:
from os.path import isdir, join

labels = [f for f in os.listdir(train_audio_path) if isdir(join(train_audio_path, f))]
print('Number of labels: ' + str(len(labels)))
print('Labels: '+ ','.join(labels))

In [ ]:
#find count of each label and plot bar graph
no_of_recordings=[]
for label in labels:
    waves = [f for f in os.listdir(train_audio_path + '/'+ label) if f.endswith('.wav')]
    no_of_recordings.append(len(waves))
    
#plot
plt.figure(figsize=(30,5))
index = np.arange(len(labels))
plt.bar(index, no_of_recordings)
plt.xlabel('Commands', fontsize=12)
plt.ylabel('No of recordings', fontsize=12)
plt.xticks(index, labels, fontsize=15, rotation=60)
plt.title('No. of recordings for each command')
plt.show()

In [ ]:
all_wave = []
all_label = []
for label in labels:
    #print(label)
    waves = [f for f in os.listdir(train_audio_path + '/'+ label) if f.endswith('.wav')]
    count =0
    for wav in waves:
        count += 1
        sample_rate, samples = wavfile.read(str(train_audio_path) + filename)
        samples = signal.resample(samples, int(new_sample_rate/sample_rate * samples.shape[0]))
        #samples, sample_rate = librosa.load(train_audio_path + '/' + label + '/' + wav, sr = 16000)
        #samples = librosa.resample(samples, sample_rate, 8000)
        if(len(samples)== 8000) : 
            all_wave.append(samples)
            all_label.append(label)
    #print(count)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y=le.fit_transform(all_label)
classes= list(le.classes_)

In [ ]:
from keras.utils import np_utils
y=np_utils.to_categorical(y, num_classes=len(labels))

In [ ]:
#all_wave = all_wave[:30000
all_wave = np.array(all_wave).reshape(-1,8000,1)
len(all_wave)

In [ ]:
from sklearn.model_selection import train_test_split
#y = y[:30000]
#x_train,x_test,y_train,y_test=train_test_split(all_wave,y,test_size=0.33)
#x_train.shape
x_train, X_valtest, y_train, y_valtest = train_test_split(all_wave,y,test_size=0.2, random_state=37,shuffle = True)
x_val, x_test, y_val, y_test = train_test_split(X_valtest,y_valtest,test_size=0.5, random_state=37)

In [ ]:
from keras.layers import Dense, Dropout, Flatten, Conv1D, Input, MaxPooling1D
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
K.clear_session()

inputs = Input(shape=(8000,1))

#First Conv1D layer
conv = Conv1D(8,13, padding='valid', activation='relu', strides=1)(inputs)
conv = MaxPooling1D(3)(conv)
conv = Dropout(0.3)(conv)

#Second Conv1D layer
conv = Conv1D(16, 11, padding='valid', activation='relu', strides=1)(conv)
conv = MaxPooling1D(3)(conv)
conv = Dropout(0.3)(conv)

#Third Conv1D layer
conv = Conv1D(32, 9, padding='valid', activation='relu', strides=1)(conv)
conv = MaxPooling1D(3)(conv)
conv = Dropout(0.3)(conv)

#Fourth Conv1D layer
conv = Conv1D(64, 7, padding='valid', activation='relu', strides=1)(conv)
conv = MaxPooling1D(3)(conv)
conv = Dropout(0.3)(conv)

#conv = Conv1D(128, 5, padding='valid', activation='relu', strides=1)(conv)
#conv = MaxPooling1D(3)(conv)
#conv = Dropout(0.3)(conv)

#Flatten layer
conv = Flatten()(conv)

#Dense Layer 1
conv = Dense(256, activation='relu')(conv)
conv = Dropout(0.3)(conv)

#Dense Layer 2
conv = Dense(128, activation='relu')(conv)
conv = Dropout(0.3)(conv)

outputs = Dense(len(labels), activation='softmax')(conv)

model = Model(inputs, outputs)
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10, min_delta=0.001) 
mc = ModelCheckpoint('best_model.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='max')

In [ ]:
history=model.fit(x_train, y_train ,callbacks=[es,mc], epochs=10, batch_size=32)